In [0]:
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)
FOLDER= '/content/gdrive/My Drive/Colab Notebooks'
os.chdir(FOLDER)
print(os.getcwd())

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks


In [0]:
!pip install bayesian-optimization

In [0]:
import pandas as pd                         # 데이터 분석 라이브러리
import numpy as np                          # 계산 라이브러리
from tqdm import tqdm                       # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
import xgboost as xgb
from sklearn.model_selection import KFold   # K-fold CV
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리
from functools import partial               # 함수 변수 고정
from keras.utils import to_categorical      # 카테고리 범주바꾸는 라이브러리
import numpy as np
import warnings                             
warnings.filterwarnings("ignore")           # 경고 문구 미표시
import random                               # random 변수 생성

Using TensorFlow backend.


In [0]:
# 데이터가 많아 5분정도 걸립니다.
train = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/competition data/startcraft_data/train.csv") 
test = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/competition data/startcraft_data/test.csv") 
sample_submission_data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/competition data/startcraft_data/sample_submission.csv") 

In [0]:
train.head()

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"


In [0]:
test.head()

,game_id,time,player,species,event,event_contents
0,38872,0.00,0,P,Camera,"at (22.25, 81.5078125)"
1,38872,0.00,1,P,Camera,"at (120.25, 153.83984375)"
2,38872,0.01,1,P,Selection,['Nexus [3100001]']
3,38872,0.01,1,P,Ability,(15E0) - TrainProbe
4,38872,0.01,1,P,AddToControlGroup,NaN


In [0]:
sample_submission_data.head()

,game_id,winner
0,38872,0
1,38873,0
2,38874,0
3,38875,0
4,38876,0


In [0]:
sample_submission_data['winner'].value_counts()

0    16787
Name: winner, dtype: int64

In [0]:
df_train = pd.DataFrame(columns=['game_id'])
df_train.game_id = train.game_id.unique()

In [0]:
df_test = pd.DataFrame(columns=['game_id'])
df_test.game_id = test.game_id.unique()

In [0]:
df_train['time'] = np.array(train[train.shift(-1).game_id != train.game_id].time)

In [0]:
df_test['time'] = np.array(test[test.shift(-1).game_id != test.game_id].time)

In [0]:
df_train.head()

,game_id,time
0,0,7.24
1,1,9.59
2,2,9.59
3,3,9.59
4,4,5.43


In [0]:
df_test.head()

,game_id,time
0,38872,7.06
1,38873,6.07
2,38874,4.41
3,38875,7.58
4,38876,2.50


In [0]:
df = train[train.player==0]
df_train['player0_species'] = np.array(df[df.shift(-1).game_id != df.game_id].species)
df = train[train.player==1]
df_train['player1_species'] = np.array(df[df.shift(-1).game_id != df.game_id].species)

In [0]:
df = test[test.player==0]
df_test['player0_species'] = np.array(df[df.shift(-1).game_id != df.game_id].species)
df = test[test.player==1]
df_test['player1_species'] = np.array(df[df.shift(-1).game_id != df.game_id].species)

In [0]:
df = train[train.player==0]
df_train['event_count_0'] = np.array(df.game_id.value_counts()[df.game_id.unique()])
df = train[train.player==1]
df_train['event_count_1'] = np.array(df.game_id.value_counts()[df.game_id.unique()])

In [0]:
df = test[test.player==0]
df_test['event_count_0'] = np.array(df.game_id.value_counts()[df.game_id.unique()])
df = test[test.player==1]
df_test['event_count_1'] = np.array(df.game_id.value_counts()[df.game_id.unique()])

In [0]:
df_test.head()

,game_id,time,player0_species,player1_species,event_count_0,event_count_1
0,38872,7.06,P,P,571,715
1,38873,6.07,Z,P,421,389
2,38874,4.41,T,T,469,824
3,38875,7.58,T,P,1223,863
4,38876,2.50,Z,Z,582,213


In [0]:
df_train.head()

,game_id,time,player0_species,player1_species,event_count_0,event_count_1
0,0,7.24,T,T,592,548
1,1,9.59,P,T,1223,1211
2,2,9.59,P,Z,851,1371
3,3,9.59,T,P,1389,1622
4,4,5.43,T,Z,966,651


In [0]:
for event in tqdm(train.event.unique()):
    df = train[(train.player==0)&(train.event==event)]
    df = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event+'_count_0'}, axis = 1)
    df['game_id']= np.array(df.index)
    df_train = pd.merge(df_train, df, on='game_id', how='left')

    df = train[(train.player==1)&(train.event==event)]
    df = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event+'_count_1'}, axis = 1)
    df['game_id']= np.array(df.index)
    df_train = pd.merge(df_train, df, on='game_id', how='left')
df_train = df_train.fillna(0)

100%|██████████| 8/8 [01:10<00:00,  8.80s/it]


In [0]:
for event in tqdm(test.event.unique()):
    df = test[(test.player==0)&(test.event==event)]
    df = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event+'_count_0'}, axis = 1)
    df['game_id']= np.array(df.index)
    df_test = pd.merge(df_test, df, on='game_id', how='left')

    df = test[(test.player==1)&(test.event==event)]
    df = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event+'_count_1'}, axis = 1)
    df['game_id']= np.array(df.index)
    df_test = pd.merge(df_test, df, on='game_id', how='left')
df_test = df_test.fillna(0)

100%|██████████| 8/8 [00:29<00:00,  3.68s/it]


In [0]:
for event in tqdm(train.event.unique()):
    df_train[event + '_delta'] = df_train[event+'_count_0'] - df_train[event+'_count_1']

100%|██████████| 8/8 [00:00<00:00, 199.38it/s]


In [0]:
for event in tqdm(test.event.unique()):
    df_test[event + '_delta'] = df_test[event+'_count_0'] - df_test[event+'_count_1']

100%|██████████| 8/8 [00:00<00:00, 848.30it/s]


In [0]:
df_train['winner'] = np.array(train[train.shift(-1).game_id != train.game_id].winner)

In [0]:
idx = df_train[df_train.player0_species=='T'].index
df_train.loc[idx, 'player0_species'] = 0
idx = df_train[df_train.player0_species=='P'].index
df_train.loc[idx, 'player0_species'] = 1
idx = df_train[df_train.player0_species=='Z'].index
df_train.loc[idx, 'player0_species'] = 2

idx = df_train[df_train.player1_species=='T'].index
df_train.loc[idx, 'player1_species'] = 0
idx = df_train[df_train.player1_species=='P'].index
df_train.loc[idx, 'player1_species'] = 1
idx = df_train[df_train.player1_species=='Z'].index
df_train.loc[idx, 'player1_species'] = 2

In [0]:
idx = df_test[df_test.player0_species=='T'].index
df_test.loc[idx, 'player0_species'] = 0
idx = df_test[df_test.player0_species=='P'].index
df_test.loc[idx, 'player0_species'] = 1
idx = df_test[df_test.player0_species=='Z'].index
df_test.loc[idx, 'player0_species'] = 2

idx = df_test[df_test.player1_species=='T'].index
df_test.loc[idx, 'player1_species'] = 0
idx = df_test[df_test.player1_species=='P'].index
df_test.loc[idx, 'player1_species'] = 1
idx = df_test[df_test.player1_species=='Z'].index
df_test.loc[idx, 'player1_species'] = 2

In [0]:
df_train.head()

,game_id,time,player0_species,player1_species,event_count_0,event_count_1,Camera_count_0,Camera_count_1,Selection_count_0,Selection_count_1,Ability_count_0,Ability_count_1,Right Click_count_0,Right Click_count_1,SetControlGroup_count_0,SetControlGroup_count_1,GetControlGroup_count_0,GetControlGroup_count_1,AddToControlGroup_count_0,AddToControlGroup_count_1,ControlGroup_count_0,ControlGroup_count_1,Camera_delta,Selection_delta,Ability_delta,Right Click_delta,SetControlGroup_delta,GetControlGroup_delta,AddToControlGroup_delta,ControlGroup_delta,winner
0,0,7.24,0,0,592,548,444.0,425.0,50.0,57.0,34.0,34.0,35.0,28.0,3.0,1.0,24.0,3.0,2.0,0.0,0.0,0.0,19.0,-7.0,0.0,7.0,2.0,21.0,2.0,0.0,1
1,1,9.59,1,0,1223,1211,627.0,858.0,186.0,116.0,77.0,67.0,160.0,131.0,10.0,8.0,162.0,31.0,1.0,0.0,0.0,0.0,-231.0,70.0,10.0,29.0,2.0,131.0,1.0,0.0,1
2,2,9.59,1,2,851,1371,413.0,725.0,90.0,232.0,69.0,85.0,160.0,204.0,14.0,9.0,99.0,109.0,6.0,5.0,0.0,2.0,-312.0,-142.0,-16.0,-44.0,5.0,-10.0,1.0,-2.0,0
3,3,9.59,0,1,1389,1622,713.0,388.0,180.0,148.0,82.0,89.0,276.0,268.0,6.0,19.0,132.0,710.0,0.0,0.0,0.0,0.0,325.0,32.0,-7.0,8.0,-13.0,-578.0,0.0,0.0,0
4,4,5.43,0,2,966,651,430.0,272.0,67.0,126.0,57.0,36.0,177.0,106.0,10.0,8.0,224.0,99.0,1.0,4.0,0.0,0.0,158.0,-59.0,21.0,71.0,2.0,125.0,-3.0,0.0,0


In [0]:
df_test.head()

,game_id,time,player0_species,player1_species,event_count_0,event_count_1,Camera_count_0,Camera_count_1,Selection_count_0,Selection_count_1,Ability_count_0,Ability_count_1,AddToControlGroup_count_0,AddToControlGroup_count_1,SetControlGroup_count_0,SetControlGroup_count_1,Right Click_count_0,Right Click_count_1,GetControlGroup_count_0,GetControlGroup_count_1,ControlGroup_count_0,ControlGroup_count_1,Camera_delta,Selection_delta,Ability_delta,AddToControlGroup_delta,SetControlGroup_delta,Right Click_delta,GetControlGroup_delta,ControlGroup_delta
0,38872,7.06,1,1,571,715,232.0,467.0,52.0,41.0,31.0,38.0,1.0,4.0,5.0,0.0,241.0,122.0,9.0,43.0,0.0,0.0,-235.0,11.0,-7.0,-3.0,5.0,119.0,-34.0,0.0
1,38873,6.07,2,1,421,389,200.0,146.0,114.0,19.0,34.0,28.0,0.0,1.0,0.0,5.0,51.0,68.0,18.0,122.0,4.0,0.0,54.0,95.0,6.0,-1.0,-5.0,-17.0,-104.0,4.0
2,38874,4.41,0,0,469,824,245.0,220.0,42.0,36.0,33.0,29.0,0.0,0.0,7.0,5.0,79.0,108.0,63.0,426.0,0.0,0.0,25.0,6.0,4.0,0.0,2.0,-29.0,-363.0,0.0
3,38875,7.58,0,1,1223,863,515.0,357.0,123.0,99.0,95.0,58.0,8.0,1.0,9.0,2.0,218.0,322.0,255.0,24.0,0.0,0.0,158.0,24.0,37.0,7.0,7.0,-104.0,231.0,0.0
4,38876,2.50,2,2,582,213,129.0,161.0,25.0,35.0,7.0,8.0,2.0,0.0,2.0,1.0,30.0,7.0,387.0,1.0,0.0,0.0,-32.0,-10.0,-1.0,2.0,1.0,23.0,386.0,0.0


In [0]:
df_train.shape

(38872, 31)

In [0]:
df_test.shape

(16787, 30)

In [0]:
x_train = df_train.iloc[:, :-1]
x_train.head()

,game_id,time,player0_species,player1_species,event_count_0,event_count_1,Camera_count_0,Camera_count_1,Selection_count_0,Selection_count_1,Ability_count_0,Ability_count_1,Right Click_count_0,Right Click_count_1,SetControlGroup_count_0,SetControlGroup_count_1,GetControlGroup_count_0,GetControlGroup_count_1,AddToControlGroup_count_0,AddToControlGroup_count_1,ControlGroup_count_0,ControlGroup_count_1,Camera_delta,Selection_delta,Ability_delta,Right Click_delta,SetControlGroup_delta,GetControlGroup_delta,AddToControlGroup_delta,ControlGroup_delta
0,0,7.24,0,0,592,548,444.0,425.0,50.0,57.0,34.0,34.0,35.0,28.0,3.0,1.0,24.0,3.0,2.0,0.0,0.0,0.0,19.0,-7.0,0.0,7.0,2.0,21.0,2.0,0.0
1,1,9.59,1,0,1223,1211,627.0,858.0,186.0,116.0,77.0,67.0,160.0,131.0,10.0,8.0,162.0,31.0,1.0,0.0,0.0,0.0,-231.0,70.0,10.0,29.0,2.0,131.0,1.0,0.0
2,2,9.59,1,2,851,1371,413.0,725.0,90.0,232.0,69.0,85.0,160.0,204.0,14.0,9.0,99.0,109.0,6.0,5.0,0.0,2.0,-312.0,-142.0,-16.0,-44.0,5.0,-10.0,1.0,-2.0
3,3,9.59,0,1,1389,1622,713.0,388.0,180.0,148.0,82.0,89.0,276.0,268.0,6.0,19.0,132.0,710.0,0.0,0.0,0.0,0.0,325.0,32.0,-7.0,8.0,-13.0,-578.0,0.0,0.0
4,4,5.43,0,2,966,651,430.0,272.0,67.0,126.0,57.0,36.0,177.0,106.0,10.0,8.0,224.0,99.0,1.0,4.0,0.0,0.0,158.0,-59.0,21.0,71.0,2.0,125.0,-3.0,0.0


In [0]:
x_train.shape

(38872, 30)

In [0]:
y_train = df_train.iloc[:, -1]
y_train.head()

0    1
1    1
2    0
3    0
4    0
Name: winner, dtype: int64

In [0]:
y_train = np.array(y_train)

In [0]:
y_train

array([1, 1, 0, ..., 0, 1, 0])

# XGBoost 적용하기

In [0]:
# XGBClassifier
# 반드시 튜닝해야할 파라미터는 max_depth / gamma

def XGB_cv( max_depth, 
            learning_rate, 
            subsample, 
            colsample_bytree, 
            reg_alpha, 
            reg_lambda, 
            gamma, 
            max_delta_step, 
            x_data=None, 
            y_data=None, 
            n_splits=5, 
            output='score'
        ):
    
    score = 0
    kf = KFold(n_splits = n_splits)
    models = []
    for train_index, valid_index in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_index], y_data[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data[valid_index]

        model = xgb.XGBClassifier(
            # booster = 'dart',
            max_depth = int(max_depth),
            # n_estimators = int(n_estimators),
            learning_rate = learning_rate,
            subsample = np.clip(subsample, 0, 1),
            # colsample_bynode = np.clip(colsample_bynode, 0, 1),
            colsample_bytree = np.clip(colsample_bytree, 0, 1),
            reg_alpha = reg_alpha,
            reg_lambda = reg_lambda,
            gamma = gamma,
            max_delta_step = max_delta_step,
        )

        model.fit(x_train, y_train)
        models.append(model)

        pred = model.predict_proba(x_valid)[:, 1]
        true = y_valid
        score += roc_auc_score(true, pred) / n_splits

    if output == 'score':
        return score
    if output == 'model':
        return models

In [0]:
# 모델 그 외 변수는 고정
var_fixed = partial(
        XGB_cv, 
        x_data=x_train, 
        y_data=y_train, 
        n_splits=5, 
        output='score'
    )

# 베이지안 최적화 범위 설정
XGBo = BayesianOptimization(
        var_fixed, 
        {
            'max_depth': (8, 512),          # ~ 500
            'learning_rate': (0.1, 0.6),    # 0.6xxx
            'subsample': (0.5, 0.9),        # 0.5 ~ 0.7
            'colsample_bytree': (0.5, 1),   # 0.5 ~ 0.8
            # 'colsample_bynode':(0, 1),
            # 'n_estimators' : (16, 1024),
            'reg_alpha' : (7, 20),          # L1 - 7 ~ 20
            'reg_lambda' : (8, 50),         # L2 - 8 ~ 45
            'max_delta_step' : (1, 25),     # 8.x
            'gamma' : (5, 20),              # 1 ~ 15
        },
        random_state = random.randrange(50000) # 유동 시드 1~50000
    )

XGBo.maximize(init_points=15, n_iter=60)    # 처음 15회 랜덤값으로 score 계산 후 60회 최적화

|   iter    |  target   | colsam... |   gamma   | learni... | max_de... | max_depth | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.641    |  0.5165   |  15.61    |  0.2778   |  19.44    |  379.4    |  16.86    |  47.84    |  0.6544   |
|  2        |  0.6429   |  0.7431   |  11.1     |  0.2601   |  6.099    |  105.6    |  20.0     |  33.82    |  0.5992   |
|  3        |  0.6491   |  0.9922   |  9.382    |  0.3212   |  19.21    |  171.7    |  8.512    |  18.57    |  0.7829   |
|  4        |  0.6459   |  0.8146   |  11.11    |  0.3454   |  22.34    |  397.8    |  12.26    |  27.95    |  0.5339   |
|  5        |  0.6453   |  0.8961   |  8.893    |  0.4713   |  3.639    |  387.5    |  16.29    |  11.36    |  0.604    |
|  6        |  0.6422   |  0.823    |  6.522    |  0.5516   |  19.06    |  71.52    |  13.16    |  44.46    |  0.6045   |
|  7        |  0.6469   

In [0]:
params = XGBo.max['params']
models = XGB_cv(
        params['max_depth'], 
        params['learning_rate'],
        params['subsample'], 
        params['colsample_bytree'],
        # params['colsample_bynode'], 
        params['reg_alpha'], 
        params['reg_lambda'],
        params['max_delta_step'],
        # params['n_estimators'],
        params['gamma'],
        x_data=x_train, 
        y_data=y_train, 
        n_splits=5, 
        output='model'
    )

In [0]:
df_test.head()

,game_id,time,player0_species,player1_species,event_count_0,event_count_1,Camera_count_0,Camera_count_1,Selection_count_0,Selection_count_1,Ability_count_0,Ability_count_1,Right Click_count_0,Right Click_count_1,SetControlGroup_count_0,SetControlGroup_count_1,GetControlGroup_count_0,GetControlGroup_count_1,AddToControlGroup_count_0,AddToControlGroup_count_1,ControlGroup_count_0,ControlGroup_count_1_x,Camera_delta,Selection_delta,Ability_delta,Right Click_delta,SetControlGroup_delta,GetControlGroup_delta,AddToControlGroup_delta,ControlGroup_delta,winner,ControlGroup_count_1_y
0,0,7.24,0,0,592,548,444.0,425.0,50.0,57.0,34.0,34.0,35.0,28.0,3.0,1.0,24.0,3.0,2.0,0.0,0.0,0.0,19.0,-7.0,0.0,7.0,2.0,21.0,2.0,0.0,1,0.0
1,1,9.59,1,0,1223,1211,627.0,858.0,186.0,116.0,77.0,67.0,160.0,131.0,10.0,8.0,162.0,31.0,1.0,0.0,0.0,0.0,-231.0,70.0,10.0,29.0,2.0,131.0,1.0,0.0,1,0.0
2,2,9.59,1,2,851,1371,413.0,725.0,90.0,232.0,69.0,85.0,160.0,204.0,14.0,9.0,99.0,109.0,6.0,5.0,0.0,2.0,-312.0,-142.0,-16.0,-44.0,5.0,-10.0,1.0,-2.0,0,0.0
3,3,9.59,0,1,1389,1622,713.0,388.0,180.0,148.0,82.0,89.0,276.0,268.0,6.0,19.0,132.0,710.0,0.0,0.0,0.0,0.0,325.0,32.0,-7.0,8.0,-13.0,-578.0,0.0,0.0,0,0.0
4,4,5.43,0,2,966,651,430.0,272.0,67.0,126.0,57.0,36.0,177.0,106.0,10.0,8.0,224.0,99.0,1.0,4.0,0.0,0.0,158.0,-59.0,21.0,71.0,2.0,125.0,-3.0,0.0,0,0.0


In [0]:
preds = []
for model in models:
    pred = model.predict_proba(x_test)[:, 1]
    preds.append(pred)
    
pred = np.mean(preds, axis=0)

submission = sample_submission_data # pd.read_csv('data/sample_submission.csv', index_col=0)
submission['winner'] = submission['winner'] + pred
submission.to_csv('/content/gdrive/My Drive/Colab Notebooks/competition data/startcraft_data/submit_0327_2.csv', index=False)
submission.head()

In [0]:
submission['winner'].value_counts()

In [0]:
submission[submission['winner'] >= 1]